<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/google_sheet_interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Use of NeqSim in Googel Sheets
#@markdown Download and import packages 
!pip install py4j
!pip install --upgrade -q gspread
!rm -rf testneqsim
!git clone https://github.com/EvenSol/NeqSim-Colab.git
!cd NeqSim-Colab
!pip install NeqSim-Colab/lib/neqsim-0.0.1-py3-none-any.whl
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

    100% |████████████████████████████████| 204kB 23.1MB/s 
Cloning into 'NeqSim-Colab'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 119 (delta 39), reused 10 (delta 3), pack-reused 43
Receiving objects: 100% (119/119), 61.91 MiB | 10.16 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Processing ./NeqSim-Colab/lib/neqsim-0.0.1-py3-none-any.whl


# How to connect and create a Google sheet

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
sheetTitle = 'a new neqsim spread sheet2'  #@param {type:"string"}
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.create(sheetTitle)

# Writing values to a google sheet

In [0]:
#@title Input to calculations { run: "auto" }
#@markdown Please input valuers for the parameters:
componentName = "methane" #@param ["methane", "ethane", "propane"]
minPressure = 1.0 #@param {type:"number"}
maxPressure = 20.0 #@param {type:"number"}

fluid1 = fluid('srk') #create a fluid using the SRK-EoS
fluid1.addComponent('propane', 1.0) #adding 1 mole methane to the fluid
fluid1.setTemperature(111.0) # setting an initial temperature of -160 deg C
fluid1.setPressure(1.0)
bubt(fluid1)
print(componentName, ' buble point temperature ', fluid1.getTemperature('C'), " at pressure ", fluid1.getPressure('bara'))
print(componentName, ' entalpy of vaporization ', (fluid1.getPhase('gas').getEnthalpy('kJ/kg')-fluid1.getPhase('oil').getEnthalpy('kJ/kg'), ' kJ/kg'))

def bubleP(pressure):
    fluid1.setPressure(pressure)
    bubt(fluid1)
    return fluid1.getTemperature('C')

pressure = np.arange(minPressure, maxPressure, 1.0)
temperature = [bubleP(P) for P in pressure]

# Open our new sheet and add some data.
worksheet = gc.open(sheetTitle).sheet1
worksheet.clear()

worksheet.update_acell('A1', "pressure [bara]")
worksheet.update_acell('B1', "temperature [C]")


cell_list = worksheet.range('A2:A'+str(pressure.size+1))
cell_list2 = worksheet.range('B2:B'+str(pressure.size+1))

i=0
for cell in cell_list:
  cell.value = pressure[i]
  i +=1

i=0
for cell in cell_list2:
  cell.value = temperature[i]
  i +=1
  
  
worksheet.update_cells(cell_list)
worksheet.update_cells(cell_list2)

#
worksheet.append_row(temperature)
worksheet.append_row(pressure.tolist())


# https://gspread.readthedocs.io/en/latest/index.html

methane  buble point temperature  -42.17772086617717  at pressure  1.0
methane  entalpy of vaporization  (303.40514081044967, ' kJ/kg')


AttributeError: ignored